# Data cleaning

### This notebook goes through the datacleaning process

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
# reading in the data frames
df_1 = pd.read_csv('../Data/Item_Data/item_desc_1.csv', index_col=[0])
df_2 = pd.read_csv('../Data/Item_Data/item_desc_2.csv', index_col=[0])
df_3 = pd.read_csv('../Data/Item_Data/item_desc_3.csv', index_col=[0])
df_4 = pd.read_csv('../Data/Item_Data/item_desc_4.csv', index_col=[0])
df_5 = pd.read_csv('../Data/Item_Data/item_desc_5.csv', index_col=[0])
df_6 = pd.read_csv('../Data/Item_Data/item_desc_1.csv', index_col=[0])
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], axis=0)

In [23]:
df.head()

,username,sold_price,designer,category,description,sub_title,image_count,size_color_cond,feedback_count,link
0,tabw8045,$110 (Sold Price),Acne Studios,Acne Studios Shorts,Ryder Bermuda wool/mohair blended shorts from ...,Ryder Bermuda Wool Shorts,5,Size: US 28 / EU 44 Color: Black Condition: New,12 Feedback,https://www.grailed.com/listings/15211509-acne...
1,kenny_v,$155 (Sold Price),Acne Studios,Acne Studios Casual Pants,"Size 46,\nAsia fitting,\nWaist- around 31-32\n...",Jager Cord trousers,10,Size: US 31 Color: Brown Condition: Gently used,9 Feedback,https://www.grailed.com/listings/19510032-acne...
2,MARKEDEU,$635 (Sold Price),Acne Studios,Acne Studios Heavy Coats,- Brand name: Acne Studios\n- Item name: SS15 ...,Acne Studios SS15 Dark Grey Charlie Jacket,4,Size: US S / EU 44-46 / 1 Color: Dark grey Co...,97 Feedback,https://www.grailed.com/listings/18387779-acne...
3,mayonaise,$85 (Sold Price),Acne Studios,Acne Studios Blazers,Great Condition\nOrange Blazer,Acne Studios Noel Blazer 40R,6,Size: 40R Color: Orange Condition: Gently used,195 Feedback,https://www.grailed.com/listings/19018436-acne...
4,aeroebogaard,$120 (Sold Price),Acne Studios,Acne Studios Sweaters & Knitwear,Marked as a large fits more like a medium or m...,Black knit Bill o reverse,3,Size: US M / EU 48-50 / 2 Color: Black Condit...,6 Feedback,https://www.grailed.com/listings/19637389-acne...


In [24]:
# checking for nans and empty strings
df.isnull().sum()

username           1225
sold_price          386
designer            536
category            384
description         384
sub_title           384
image_count           0
size_color_cond     384
feedback_count     6071
link                  0
dtype: int64

In [25]:
# checking datatypes
df.dtypes

username           object
sold_price         object
designer           object
category           object
description        object
sub_title          object
image_count         int64
size_color_cond    object
feedback_count     object
link               object
dtype: object

## Cleaning up the feedback_count column

1. making column a float
2. replacing nan with 0

In [27]:
df['feedback_count'] = df['feedback_count'].str.extract('(\d+)').astype(float)

In [28]:
# changing missing feedback_count to zero instead of an empty string
df['feedback_count'] = df['feedback_count'].fillna(0)

In [29]:
df['feedback_count'].value_counts()

1.0       6219
0.0       6071
2.0       5077
3.0       4709
4.0       3934
          ... 
1181.0       1
1320.0       1
741.0        1
679.0        1
648.0        1
Name: feedback_count, Length: 1193, dtype: int64

## Dropping Remaining NANs

decided that there wasnt much of a way to fix these values and there were so few overall.

In [30]:
df.dropna(inplace=True)

In [31]:
df.isnull().sum()

username           0
sold_price         0
designer           0
category           0
description        0
sub_title          0
image_count        0
size_color_cond    0
feedback_count     0
link               0
dtype: int64

## Cleaning up sold_price column

1. cast column as a float
2. check for an zeros or negative numbers to remove

In [32]:
# changing sold_price to a float
df['sold_price'] = df['sold_price'].str.extract('(\d+)').astype(float)
df['sold_price']

0        110.0
1        155.0
2        635.0
3         85.0
4        120.0
         ...  
29864    298.0
29865    100.0
29866    350.0
29867    260.0
29868     60.0
Name: sold_price, Length: 194369, dtype: float64

In [33]:
# checking for sold_price less than or equal to zero
df[df['sold_price'] <= 0]
# no values to remove

,username,sold_price,designer,category,description,sub_title,image_count,size_color_cond,feedback_count,link


## Cleaing up category column

1. removing designer name from category
2. drop rows if category contains less than 10

In [34]:
# reseting the index given it had values dropped
df.index = range(len(df))

In [35]:
# making sure category is a string
df['category'] = df['category'].astype(str)

In [36]:
# making sure designer is a string
df['designer'] = df['designer'].astype(str)

In [37]:
# function to replace designers in category column with empty string leaving only the item category
def replace_designer(row):
    words = df['designer'][row].split() # splitting the designer into multple words incase multiple designers are present
    for word in words:
        if word in df['category'][row]:
            df['category'][row] = df['category'][row].replace(word,'') # goes through each word and replaces with empty string if present
    return df['category'][row].strip()

In [38]:
# using function on all data to remove designer from category
for i in range(len(df['category'])):
    replace_designer(i)

<ipython-input-37-8bd2803f4eaa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'][row] = df['category'][row].replace(word,'') # goes through each word and replaces with empty string if present


In [39]:
df['category']= df['category'].map(lambda x: x.strip())

In [40]:
# dropping all rows if category count is less than 10
df = df.groupby('category').filter(lambda x : len(x)>10)

## Cleaning up size_color_condition column

1. create a column for size
2. create a column for color
3. create a column for condition
4. drop all rows that all 3 columns could not be created

In [41]:
# resetting the index
df.index = range(len(df))

In [42]:
# extracting the size into a new size column
def get_size(row):
    words = df['size_color_cond'][row]
    try:
        df['size'][row] = words[words.find('Size:'):words.find('Color:')].replace('Size:', '').strip() # will find all things after size
    except: 
        df['size'][row] = np.nan
    return df['size'][row]

In [43]:
# applying get_size to all rows
df['size'] = 0
for i in range(len(df['size_color_cond'])):
    get_size(i)

<ipython-input-42-c7affa5dbf5f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'][row] = words[words.find('Size:'):words.find('Color:')].replace('Size:', '').strip() # will find all things after size
C:\Users\alber\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [44]:
# extracting the color into a new color column
def get_color(row):
    words = df['size_color_cond'][row] # splitting the designer into multple words incase multiple designers are present
    try:
        df['color'][row] = words[words.find('Color:'):words.find('Condition:')].replace('Color:', '').strip()
    except:
        df['color'][row] = np.nan
    return df['color'][row]

In [45]:
# applying get_color to all rows
df['color'] = 'none'
for i in range(len(df['size_color_cond'])):
    get_color(i)

<ipython-input-44-1682ea72a615>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['color'][row] = words[words.find('Color:'):words.find('Condition:')].replace('Color:', '').strip()


In [46]:
# extracting the condition into a new condition column
def get_condition(row):
    words = df['size_color_cond'][row] 
    try:
        df['condition'][row] = words[words.find('Condition:')::].replace('Condition:', '').strip()
    except:
        df['condition'][row] = np.nan
    return df['condition'][row]

In [47]:
# applying get_condition to all rows
df['condition'] = 'none'
for i in range(len(df['size_color_cond'])):
    get_condition(i)

<ipython-input-46-5d5ba7cac5dd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['condition'][row] = words[words.find('Condition:')::].replace('Condition:', '').strip()


In [48]:
# replacing empty strings with nan
df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [49]:
# dropping rows that did not have all 3 categories, size color and condition
df.dropna(inplace=True)

# Cleaning condition column

1. removing outlier conditions

In [50]:
# dropping all rows of condition count less than 100
df = df.groupby('condition').filter(lambda x : len(x)>100)

# Cleaning color column

1. removing outlier colors

In [51]:
df = df.groupby('color').filter(lambda x : len(x)>100)

# Cleaning size column

1. removing outlier sizes

In [52]:
df = df.groupby('size').filter(lambda x : len(x)>20)

## Cleaning up description column

1. remove new lines (\n) from description

In [53]:
# replacing \n with a space
df['description'] = df['description'].replace('\n',' ',regex=True)

## Cleaning up designer column

1. Dealing with collobrations between designers
2. Drop rows where designer count is less than 25

In [54]:
# resetting the index
df.index = range(len(df))

In [55]:
# function to change coloborations into one designer
def one_designer(row):
    word = df['designer'][row]
    words = word.split() 
    if 'x' in words:
        df['designer'][row] = df['designer'][row][:word.index('x')].strip() # if x is in the designer name will only return first designer
    return df['designer'][row]

In [56]:
# applyng one_designer to all rows
for i in range(len(df['designer'])):
    one_designer(i)

<ipython-input-55-0da5feab6913>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['designer'][row] = df['designer'][row][:word.index('x')].strip() # if x is in the designer name will only return first designer


In [57]:
df['designer'].value_counts()

Vintage              26984
Nike                  3126
Supreme               1524
Adidas                1363
Japanese Brand         908
                     ...  
Uniform E                1
Universal Studios        1
Ts(S)                    1
Ripcurl                  1
Shockoe Denim            1
Name: designer, Length: 2263, dtype: int64

In [58]:
# dropping all rows of designer count less than 25
df = df.groupby('designer').filter(lambda x : len(x)>25)

## Final Dataframe

In [59]:
# dropping duplicates
df.drop_duplicates(inplace=True)

In [65]:
# exporting final data
df.to_csv('modeling_data.csv', index=False)